# rechunker sandbox

In [22]:
import intake
import xarray as xr
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from dask.distributed import Client, LocalCluster
import dask
import datetime
import zarr

In [2]:
cluster=LocalCluster(n_workers=28,processes=True,threads_per_worker=1)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 251.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33221,Workers: 28
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.18 GiB
Comm: tcp://127.0.0.1:34123,Total threads: 1
Dashboard: /proxy/41335/status,Memory: 8.97 GiB
Nanny: tcp://127.0.0.1:35297,


In [3]:
def keep_only_selected_vars(ds, vars_to_keep=['temp','Time','st_ocean','yt_ocean','xt_ocean']):
    # Calculate which variables to drop by finding the difference
    # between all variables in the dataset and the ones you want to keep
    vars_to_drop = set(ds.variables) - set(vars_to_keep)
    return ds.drop_vars(list(vars_to_drop))

In [18]:
xarray_open_kwargs = {"Time": 1, "st_ocean": 1, "xt_ocean": 3600, "yt_ocean": 1500}
ds = xr.open_mfdataset('/g/data/gb6/BRAN/BRAN2020/daily/ocean_temp_*.nc',parallel=True,chunks=xarray_open_kwargs,preprocess=keep_only_selected_vars)
ds

<xarray.Dataset> Size: 402GB
Dimensions:   (xt_ocean: 3600, yt_ocean: 1500, st_ocean: 51, Time: 365)
Coordinates:
  * xt_ocean  (xt_ocean) float64 29kB 0.05 0.15 0.25 0.35 ... 359.8 359.9 360.0
  * yt_ocean  (yt_ocean) float64 12kB -74.95 -74.85 -74.75 ... 74.75 74.85 74.95
  * st_ocean  (st_ocean) float64 408B 2.5 7.5 12.5 ... 3.603e+03 4.509e+03
  * Time      (Time) datetime64[ns] 3kB 2023-01-01T12:00:00 ... 2023-12-31T12...
Data variables:
    temp      (Time, st_ocean, yt_ocean, xt_ocean) float32 402GB dask.array<chunksize=(1, 1, 1500, 3600), meta=np.ndarray>
Attributes:
    filename:           TMP/ocean_temp_2023_01_01.nc.0000
    NumFilesInSet:      20
    grid_type:          regular
    history:            Mon May 15 10:09:52 2023: ncrcat -4 --dfl_lvl 1 --cnk...
    NCO:                netCDF Operators version 5.0.5 (Homepage = http://nco...
    title:              BRAN2020
    catalogue_doi_url:  http://dx.doi.org/10.25914/6009627c7af03
    acknowledgement:    BRAN is made freely available by CSIRO Bluelink and i...

In [6]:
BRAN2020_ard_path = '/scratch/es60/ard/reanalysis/BRAN2020/ARD/'
ard_file_ID = 'TEST-temp-chunk1-1-1500-3600-v26042024.zarr'

In [39]:
%%time
ds.astype('float32').to_zarr(BRAN2020_ard_path+ard_file_ID,consolidated=True)

CPU times: user 1min 27s, sys: 7.91 s, total: 1min 35s
Wall time: 2min 23s


In [7]:
ds_1_1_1500_3600 = xr.open_zarr(BRAN2020_ard_path+ard_file_ID,consolidated=True)

In [8]:
ds_1_1_1500_3600

<xarray.Dataset> Size: 402GB
Dimensions:   (Time: 365, st_ocean: 51, yt_ocean: 1500, xt_ocean: 3600)
Coordinates:
  * Time      (Time) datetime64[ns] 3kB 2023-01-01T12:00:00 ... 2023-12-31T12...
  * st_ocean  (st_ocean) float64 408B 2.5 7.5 12.5 ... 3.603e+03 4.509e+03
  * xt_ocean  (xt_ocean) float64 29kB 0.05 0.15 0.25 0.35 ... 359.8 359.9 360.0
  * yt_ocean  (yt_ocean) float64 12kB -74.95 -74.85 -74.75 ... 74.75 74.85 74.95
Data variables:
    temp      (Time, st_ocean, yt_ocean, xt_ocean) float32 402GB dask.array<chunksize=(1, 1, 1500, 3600), meta=np.ndarray>
Attributes:
    NCO:                netCDF Operators version 5.0.5 (Homepage = http://nco...
    NumFilesInSet:      20
    acknowledgement:    BRAN is made freely available by CSIRO Bluelink and i...
    catalogue_doi_url:  http://dx.doi.org/10.25914/6009627c7af03
    filename:           TMP/ocean_temp_2023_01_01.nc.0000
    grid_type:          regular
    history:            Mon May 15 10:09:52 2023: ncrcat -4 --dfl_lvl 1 --cnk...
    title:              BRAN2020

In [9]:
def remove_zarr_encoding(DS):
    for var in DS:
        DS[var].encoding = {}

    for coord in DS.coords:
        DS[coord].encoding = {}
    return DS

In [14]:
chunking_dict={'Time':-1,'st_ocean':1,'xt_ocean':150,'yt_ocean':150}

In [11]:
var='temp'

In [15]:
chunking_string = 'chunks_'+''.join(str(key) + str(value) for key, value in chunking_dict.items())
ard_rcTime_file_ID = 'TEST-'+var+'-'+chunking_string+'-v25042024.zarr'

In [19]:
from rechunker import rechunk
!rm -rf /scratch/es60/ard/reanalysis/BRAN2020/ARD/temp_store/*
# Define target chunking
target_chunks = chunking_dict
target_store = BRAN2020_ard_path+ard_rcTime_file_ID
temp_store = BRAN2020_ard_path+'temp_store'  # Optional based on dataset size

# Execute rechunking
rechunk_plan = rechunk(ds_1_1_1500_3600, target_chunks, target_store=target_store, temp_store=temp_store,max_mem='2GB')

In [16]:
ds_1_1_1500_3600.chunk(chunking_dict)

<xarray.Dataset> Size: 402GB
Dimensions:   (Time: 365, st_ocean: 51, yt_ocean: 1500, xt_ocean: 3600)
Coordinates:
  * Time      (Time) datetime64[ns] 3kB 2023-01-01T12:00:00 ... 2023-12-31T12...
  * st_ocean  (st_ocean) float64 408B 2.5 7.5 12.5 ... 3.603e+03 4.509e+03
  * xt_ocean  (xt_ocean) float64 29kB 0.05 0.15 0.25 0.35 ... 359.8 359.9 360.0
  * yt_ocean  (yt_ocean) float64 12kB -74.95 -74.85 -74.75 ... 74.75 74.85 74.95
Data variables:
    temp      (Time, st_ocean, yt_ocean, xt_ocean) float32 402GB dask.array<chunksize=(365, 1, 150, 150), meta=np.ndarray>
Attributes:
    NCO:                netCDF Operators version 5.0.5 (Homepage = http://nco...
    NumFilesInSet:      20
    acknowledgement:    BRAN is made freely available by CSIRO Bluelink and i...
    catalogue_doi_url:  http://dx.doi.org/10.25914/6009627c7af03
    filename:           TMP/ocean_temp_2023_01_01.nc.0000
    grid_type:          regular
    history:            Mon May 15 10:09:52 2023: ncrcat -4 --dfl_lvl 1 --cnk...
    title:              BRAN2020

In [20]:
%%time
rechunk_plan.execute()

CPU times: user 3min 24s, sys: 28 s, total: 3min 52s
Wall time: 7min 53s


<zarr.hierarchy.Group '/'>

In [24]:
zarr.consolidate_metadata(target_store)
ds_365_1_150_150 = xr.open_zarr(target_store,consolidated=True)

In [25]:
ds_365_1_150_150

<xarray.Dataset> Size: 402GB
Dimensions:   (Time: 365, st_ocean: 51, yt_ocean: 1500, xt_ocean: 3600)
Coordinates:
  * Time      (Time) datetime64[ns] 3kB 2023-01-01T12:00:00 ... 2023-12-31T12...
  * st_ocean  (st_ocean) float64 408B 2.5 7.5 12.5 ... 3.603e+03 4.509e+03
  * xt_ocean  (xt_ocean) float64 29kB 0.05 0.15 0.25 0.35 ... 359.8 359.9 360.0
  * yt_ocean  (yt_ocean) float64 12kB -74.95 -74.85 -74.75 ... 74.75 74.85 74.95
Data variables:
    temp      (Time, st_ocean, yt_ocean, xt_ocean) float32 402GB dask.array<chunksize=(365, 1, 150, 150), meta=np.ndarray>
Attributes:
    NCO:                netCDF Operators version 5.0.5 (Homepage = http://nco...
    NumFilesInSet:      20
    acknowledgement:    BRAN is made freely available by CSIRO Bluelink and i...
    catalogue_doi_url:  http://dx.doi.org/10.25914/6009627c7af03
    filename:           TMP/ocean_temp_2023_01_01.nc.0000
    grid_type:          regular
    history:            Mon May 15 10:09:52 2023: ncrcat -4 --dfl_lvl 1 --cnk...
    title:              BRAN2020

In [26]:
chunking_dict={'Time':-1,'st_ocean':1,'xt_ocean':150,'yt_ocean':150}
ds_365_1_150_150.chunk(chunking_dict)

<xarray.Dataset> Size: 402GB
Dimensions:   (Time: 365, st_ocean: 51, yt_ocean: 1500, xt_ocean: 3600)
Coordinates:
  * Time      (Time) datetime64[ns] 3kB 2023-01-01T12:00:00 ... 2023-12-31T12...
  * st_ocean  (st_ocean) float64 408B 2.5 7.5 12.5 ... 3.603e+03 4.509e+03
  * xt_ocean  (xt_ocean) float64 29kB 0.05 0.15 0.25 0.35 ... 359.8 359.9 360.0
  * yt_ocean  (yt_ocean) float64 12kB -74.95 -74.85 -74.75 ... 74.75 74.85 74.95
Data variables:
    temp      (Time, st_ocean, yt_ocean, xt_ocean) float32 402GB dask.array<chunksize=(365, 1, 150, 150), meta=np.ndarray>
Attributes:
    NCO:                netCDF Operators version 5.0.5 (Homepage = http://nco...
    NumFilesInSet:      20
    acknowledgement:    BRAN is made freely available by CSIRO Bluelink and i...
    catalogue_doi_url:  http://dx.doi.org/10.25914/6009627c7af03
    filename:           TMP/ocean_temp_2023_01_01.nc.0000
    grid_type:          regular
    history:            Mon May 15 10:09:52 2023: ncrcat -4 --dfl_lvl 1 --cnk...
    title:              BRAN2020